# Final Project
## Tyler Baker and Rathish Parayil Sasidharan

Your project should incorporate one or both of the two main themes of this course: network analysis and text processing. You need to show all of your work in a coherent workflow, and in a reproducible format, such as an IPython Notebook or an R Markdown document. If you are building a model or models, explain how you evaluate the “goodness” of the chosen model and parameters. 

## Introduction


### Setting the Environment

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import nltk
import random
import networkx.algorithms.bipartite as bipartite
import matplotlib.pyplot as plt
import re

### The Data
We got our data from <a href="https://www.kaggle.com/datasets/d4rklucif3r/restaurant-reviews">Kaggle</a>. The data consists of two columns: reviews and scores.

In [4]:
data_raw = pd.read_table("https://raw.githubusercontent.com/tylerbaker01/Data-620/main/Restaurant_Reviews.tsv")

In [5]:
data_raw.columns = ["review", "score"]

In [6]:
data_raw.tail()

,review,score
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0
999,"Then, as if I hadn't wasted enough of my life ...",0


We want to add restaurant IDs and Customer Ids. In order to have repitition we must insure that the amount of reviews > the amount of users > the amount of restaurants. Note that there are 1000 reviews. Let's say that there are 600 users and 100 restaurants. We chose these numbers at random.

In [24]:
users_list = []
for i in range(100):
    users_list.append(f"Customer {i}")
users_list = users_list*100

In [25]:
restaurants_list = []
for i in range(20):
    restaurants_list.append(f"Restaurant {i}")
restaurants_list = restaurants_list *500

Now we have made a list of 600 users and 100 restaurants. Next we must make a list of length 1000 for each where each index is randomly selected from the lists above.

In [26]:
random.seed(97)
users= []
for i in range(1000):
    users.append(random.choice(users_list)) 

In [27]:
restaurants = []
for i in range(1000):
    restaurants.append(random.choice(restaurants_list))

In [28]:
data_raw["user_id"] = users
data_raw["restaurant_id"] = restaurants

In [29]:
df = data_raw
df.head()

,review,score,user_id,restaurant_id
0,Wow... Loved this place.,1,Customer 87,Restaurant 13
1,Crust is not good.,0,Customer 28,Restaurant 2
2,Not tasty and the texture was just nasty.,0,Customer 95,Restaurant 16
3,Stopped by during the late May bank holiday of...,1,Customer 34,Restaurant 17
4,The selection on the menu was great and so wer...,1,Customer 10,Restaurant 17


We have now created our data frame.

## Part 2 - NLP
### Analyze Review Data
Our goal here is to analyze the review data.

In [14]:
df_nlp = df

In [15]:
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS

In [19]:
text = df_nlp[['review']]

,review
0,Wow... Loved this place.
1,Crust is not good.
2,Not tasty and the texture was just nasty.
3,Stopped by during the late May bank holiday of...
4,The selection on the menu was great and so wer...


In [21]:
# Create a dataset based on  just the review column.
text = df_nlp[['review']]

class lemmatizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, df):
        return [self.wnl.lemmatize(word) for word in word_tokenize(df)]
    
vectorizer = CountVectorizer(max_features=100, max_df=0.95, min_df=0.1, ngram_range=(1,2),
                            tokenizer=lemmatizer(), lowercase=True, stop_words='english', token_pattern = r'\w+')

vectorizer.fit(text["review"])
count_vector = vectorizer.transform(text["review"])
count_vector

NameError: name 'WordNetLemmatizer' is not defined

In [ ]:
# Filter rows with column
data = dataset.loc[dataset['Review Text'].notnull(),:]

# Apply uni- and bigram vectorizer
class lemmatizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, df):
        return [self.wnl.lemmatize(word) for word in word_tokenize(df)]

vectorizer = CountVectorizer(max_features=100, max_df=0.95, min_df=0.1, ngram_range=(1,2),
                             tokenizer=lemmatizer(), lowercase=True, stop_words='english',
                             token_pattern = r'\w+')

vectorizer.fit(data['Review Text'])
count_vector = vectorizer.transform(data['Review Text'])
count_vector

### Rating Prediction
The goal here is to take the review data and split it into a training set and a test set. Then we train the training set until we can attempt to predict the test set. We want each review to be either rated as: Good, Fair, or Bad. 